In [1]:
import numpy as np
import matplotlib.pyplot as plt
from helper import *
import scipy.sparse as sp
from test import load_data_sparse
from SGD import * 
from ALS import *
from Visualization import cv_result
from sklearn.linear_model import LinearRegression
from crossval import *

In [2]:
pred_lst = []

In [3]:
data_name = "data/47b05e70-6076-44e8-96da-2530dc2187de_data_train.csv"

ratings, pandas = load_data_sparse(data_name, False)

print(pandas.shape) # (1176952, 3)

stop_criterion = 1e-4
prediction = []
pred_lst = []

(1176952, 3)


In [4]:
# SGD
nb_feature = 40
lambda_ = 0.01
pred_SGD, _, _ = matrix_factorization_SGD_CV(ratings, nb_feature, lambda_, lambda_, stop_criterion)


learn the matrix factorization using SGD...


In [5]:
print(type(pred_SGD))
pred_lst.append(pred_SGD)
print(len(pred_lst))

<class 'numpy.ndarray'>
1


In [6]:
# ALS
nb_feature = 40
lambda_ = 0.1
pred_ALS, _, _ = ALS_CV(ratings, nb_feature, lambda_, lambda_, stop_criterion)


learn the matrix factorization using ALS...


In [7]:
pred_lst.append(pred_ALS)
print(len(pred_lst))

2


In [8]:
pred_lst_2 = pred_lst.copy()

In [9]:
len(pred_lst_2)

2

In [10]:
simple_models = False#True#False

In [11]:
if simple_models:
    # simple_models
    from simple_models import *
    print("learn global mean")
    pred_globalmean =  global_mean(ratings, 0)
    print(pred_globalmean.shape)
    pred_lst.append(pred_globalmean)

    print("learn user mean")
    pred_usermean = user_mean(ratings, 0)
    print(pred_usermean.shape)
    pred_lst.append(pred_usermean)

    print("learn item mean")
    pred_itemmean = baseline_item_mean(ratings, 0)
    print(pred_itemmean.shape)
    pred_lst.append(pred_itemmean)

In [12]:
def prepareBlending(ratings, list_pred):
    # determine position of given ratings
    nonzero_row, nonzero_col = ratings.nonzero()
    nonzero_train = list(zip(nonzero_row, nonzero_col))
    
    prediction = []
    prediction_model = []
    
    for pred_id, pred in enumerate(list_pred):
        print(pred_id)
        
        print(pred.shape)
        
        pred_tmp = []
        for i in nonzero_train:
            pred_tmp.append(pred.item(i))

        print(len(pred_tmp)) # (1176952)
        #print(type(pred_tmp))
        
        prediction_model.append(pred_tmp)
    
        if pred_id == 0:
            #prediction_model = pred_tmp
            prediction = pred_tmp
        else:
            prediction = np.vstack((prediction,pred_tmp))
            
        
        print(len(prediction))
        print(type(prediction))
        
    return prediction_model, prediction

In [13]:
prediction_m, prediction = prepareBlending(ratings, pred_lst)
print(prediction.shape)
print(len(prediction_m))

0
(10000, 1000)
1176952
1176952
<class 'list'>
1
(10000, 1000)
1176952
2
<class 'numpy.ndarray'>
(2, 1176952)
2


In [15]:
print(len(prediction_m))

2


## Blending

In [16]:
#pandas.head()
y = pandas['Rating']
y.head()

0    4
1    3
2    4
3    3
4    5
Name: Rating, dtype: int64

In [17]:
X = np.transpose(prediction)
X[0:5]

array([[3.80625846, 3.95806939],
       [3.91891856, 4.00397926],
       [3.40215908, 3.50772056],
       [4.42462316, 3.96513662],
       [4.1365401 , 3.58340213]])

#### Linear Regression

In [71]:
#clf = LinearRegression()
#clf = clf.fit(X, y)
#print('Weights of the different models:', clf.coef_)
#clf.score(X, y)

#### Ridge Regression

In [18]:
from sklearn.linear_model import Ridge
linreg = Ridge(alpha=0.1, fit_intercept=False)
linreg.fit(X, y)
linreg.coef_

array([0.4417687 , 0.55460439])

#### Random Forest

In [19]:
from sklearn.ensemble import RandomForestClassifier
rf_classifier = RandomForestClassifier(random_state=1)
rf_classifier.fit(X, y)
rf_classifier.coef_

AttributeError: 'RandomForestClassifier' object has no attribute 'coef_'

In [ ]:
from sklearn.ensemble import RandomForestRegressor
rf_regressor = RandomForestRegressor()
rf_regressor.fit(x_train, y_train)

In [74]:
linreg.score(X, y)

9.819131762123234e-06

In [55]:
# compute MSE - Blending
sum((np.round(X.dot(linreg.coef_)) - y)**2)/y.shape[0]

1.59771426532263

In [56]:
# compute MSE - SGD
sum(np.round(prediction_m[0] - y)**2)/y.shape[0]

1.6271810575112664

In [57]:
# compute MSE - ALS
sum(np.round(prediction_m[1] - y)**2)/y.shape[0]

1.6167141905532256

## Visual comparison of the results

In [58]:
print(X[0:10])

[[3.80625846 3.95806939]
 [3.91891856 4.00397926]
 [3.40215908 3.50772056]
 [4.42462316 3.96513662]
 [4.1365401  3.58340213]
 [4.49552297 4.50655337]
 [4.08698875 4.04294664]
 [4.35256245 4.04264779]
 [3.66249213 3.48697791]
 [3.81558051 3.29708151]]


In [59]:
print(y[0:10])

0    4
1    3
2    4
3    3
4    5
5    4
6    3
7    3
8    2
9    5
Name: Rating, dtype: int64


In [60]:
# final prediction
result = X.dot(linreg.coef_)
print(result[0:10])

[3.87664847 3.95187999 3.44836458 4.15374216 3.81476446 4.48533558
 4.04773962 4.16489603 3.55186761 3.51417989]


## Make the final prediction for submission

In [62]:
if simple_models:
    result = pred_SGD * linreg.coef_[0] + pred_ALS * linreg.coef_[1] + pred_globalmean * linreg.coef_[2] + pred_usermean * linreg.coef_[3] + pred_itemmean * linreg.coef_[4]
else:
    result = pred_SGD * linreg.coef_[0] + pred_ALS * linreg.coef_[1]
        
    

In [63]:
result.shape

(10000, 1000)

In [64]:
if simple_models:
    create_submission_from_prediction(result, "out5-0.1.csv")
else:
    create_submission_from_prediction(result, "out2-40-0.1.csv")

In [65]:
create_submission_from_prediction(pred_ALS, "out-onlyALS-40.csv")